## Chapter 4: Quantum Circuits - Cirq

In [9]:
import cirq
import numpy as np

In [10]:
print(cirq.google.Bristlecone)

                                             (0, 5)────(0, 6)
                                             │         │
                                             │         │
                                    (1, 4)───(1, 5)────(1, 6)────(1, 7)
                                    │        │         │         │
                                    │        │         │         │
                           (2, 3)───(2, 4)───(2, 5)────(2, 6)────(2, 7)───(2, 8)
                           │        │        │         │         │        │
                           │        │        │         │         │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)────(3, 6)────(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │         │         │        │        │
                  │        │        │        │         │         │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)────(4, 6)────(4, 7)───(4, 8)───(4, 9)───(4, 10)
         │        │      

In [43]:
import warnings
warnings.filterwarnings('ignore')
# Get a qubit and a circuit
qbit = cirq.LineQubit(0)
circuit = cirq.Circuit()
# Add an X gate: acts like the Pauli Matrix sigma_x
circuit.append(cirq.X(qbit))

In [44]:
# Run a simple simulation that extracts the wavefunction of this state
sim = cirq.Simulator()
result = sim.simulate(circuit)
print("\nBloch Sphere of the qubit in the final state:")
state = cirq.bloch_vector_from_state_vector(result.final_state,0)
print("x: ", np.around(state[0], 4), " y: ", np.around(state[1], 4),
" z: ", np.around(state[2], 4))


Bloch Sphere of the qubit in the final state:
x:  0.0  y:  0.0  z:  -1.0


In [45]:
# Add a measurement at the end of the circuit:
circuit.append(cirq.measure(qbit, key="Final state"))
# Display the circuit:
print("\nCircuit:")
print(circuit)


Circuit:
0: ───X───M('Final state')───


In [46]:
# Invoke the Cirq quantum simulator to execute the circuit:
simulator = cirq.Simulator()
# Simulate the circuit several times:
result = simulator.run(circuit, repetitions=10)

In [47]:
# Print the results:
print("\nResults of 10 trials:")
print(result)


Results of 10 trials:
Final state=1111111111


## Bell States

In [48]:
# Get two qubits and a circuit
qubit = [cirq.LineQubit(x) for x in range(2)]
circuit = cirq.Circuit()
# Add a Hadamard gate to qubit 0, then a CNOT gate from qubit 0 to qubit 1:
circuit.append([cirq.H(qubit[0]),
cirq.CNOT(qubit[0], qubit[1])])
# Run a simple simulation that extracts the actual final states
sim = cirq.Simulator()
result = sim.simulate(circuit)
print("\nBloch Sphere of the qubit 0 in the final state:")
state = cirq.bloch_vector_from_state_vector(result.final_state,0)
print("x: ", np.around(state[0], 4), " y: ", np.around(state[1], 4),
" z: ", np.around(state[2], 4))
print("\nBloch Sphere of the qubit 1 in the final state:")
state = cirq.bloch_vector_from_state_vector(result.final_state,1)
print("x: ", np.around(state[0], 4), " y: ", np.around(state[1], 4),
" z: ", np.around(state[2], 4))


Bloch Sphere of the qubit 0 in the final state:
x:  0.0  y:  0.0  z:  0.0

Bloch Sphere of the qubit 1 in the final state:
x:  0.0  y:  0.0  z:  0.0


In [49]:
# Add a measurement at the end of the circuit:
circuit.append(cirq.measure(*qubit, key="Final state"))
# Display the circuit:
print("\nCircuit:")
print(circuit)


Circuit:
0: ───H───@───M('Final state')───
          │   │
1: ───────X───M──────────────────


In [50]:
# Invoke the Cirq quantum simulator to execute the circuit:
simulator = cirq.Simulator()
# Simulate the circuit several times:
result = simulator.run(circuit, repetitions=10)
# Print the results:
print("\nResults:")
print(result)


Results:
Final state=1000101001, 1000101001


## Quantum Teleportation

Quantum teleportation is a process by which a quantum qubit state can be transmitted by sending only two classical bits of information. This option works by distributing entangled quantum particles. These entangled particles, once distributed in a controlled manner between two ends, create virtual channels. 

In [51]:
import random
# Define three qubits: msg = qubit[0], qalice = qubit[1], qbob = qubit[2]
qubit=[0]*(3)
qubit[0] = cirq.NamedQubit('msg')
qubit[1] = cirq.NamedQubit('qalice')
qubit[2] = cirq.NamedQubit('qbob')
circuit = cirq.Circuit()
# Create a Bell state entangled pair to be shared between Alice and Bob.
circuit.append([cirq.H(qubit[1]), cirq.CNOT(qubit[1], qubit[2])])
# Creates a random state for the Message.
ranX = random.random()
ranY = random.random()
circuit.append([cirq.X(qubit[0])**ranX, cirq.Y(qubit[0])**ranY])

In [52]:
# Unitary operator rotating the two-qubit basis of the Message and Alice's entangled qubit;
# rotates the Bell state basis to the computational basis:
circuit.append([cirq.CNOT(qubit[0], qubit[1]), cirq.H(qubit[0])])
# Combining now with a measurment in the computational basis,
# we effectively have projected this two-qubit state onto one of the four states of
# the Bell state basis:
circuit.append(cirq.measure(qubit[0], qubit[1]))

In [53]:
# Use the two classical bits from the Bell measurement to recover the
# original quantum Message on Bob's entangled qubit.
circuit.append([cirq.CNOT(qubit[1], qubit[2]), cirq.CZ(qubit[0], qubit[2])])
print("Circuit:")
print(circuit)

Circuit:
msg: ──────X^0.564───Y^0.54───@───H───M───────@───
                              │       │       │
qalice: ───H─────────@────────X───────M───@───┼───
                     │                    │   │
qbob: ───────────────X────────────────────X───@───


In [54]:
sim = cirq.Simulator()
# Run a simple simulation that applies the random X and Y gates that
# create our message.
q0 = cirq.LineQubit(0)
message = sim.simulate(cirq.Circuit([cirq.X(q0)**ranX, cirq.Y(q0)**ranY]))
print("\nBloch Sphere of the Message qubit in the initial state:")
expected = cirq.bloch_vector_from_state_vector(message.final_state,0)
print("x: ", np.around(expected[0], 4), " y: ", np.around(expected[1], 4),
" z: ", np.around(expected[2], 4))


Bloch Sphere of the Message qubit in the initial state:
x:  -0.1992  y:  -0.9796  z:  0.0252


In [55]:
# Records the final state of the simulation.
final_results = sim.simulate(circuit)
print("\nBloch Sphere of Bob's qubit in the final state:")
teleported = cirq.bloch_vector_from_state_vector(
final_results.final_state, 2)
print("x: ", np.around(teleported[0], 4), " y: ",
np.around(teleported[1], 4), " z: ", np.around(teleported[2], 4))
print("\nBloch Sphere of the Message qubit in the final state:")
message_final = cirq.bloch_vector_from_state_vector(
final_results.final_state, 0)
print("x: ", np.around(message_final[0], 4), " y: ", np.around(message_final[1], 4), " z: ", np.around(message_final[2], 4))


Bloch Sphere of Bob's qubit in the final state:
x:  -0.1992  y:  -0.9796  z:  0.0252

Bloch Sphere of the Message qubit in the final state:
x:  0.0  y:  0.0  z:  -1.0
